In [ ]:
#!pip install cloud-tpu-client==0.10 torch==1.11.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.11-cp37-cp37m-linux_x86_64.whl
!pip install transformers
!pip install datasets
!pip install sentencepiece

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from ast import arg
from transformers import AutoModelForSequenceClassification
from datasets.dataset_dict import DatasetDict
from datasets import load_dataset
from datasets import load_metric
from transformers import TrainingArguments, Trainer
import torch
import numpy as np
from torch.utils.data import DataLoader
import os
from tqdm import tqdm
import torch.nn as nn
import torch
import os
#import torch_xla
#import torch_xla.core.xla_model as xm
#import torch_xla.distributed.xla_multiprocessing as xmp

In [ ]:
if not os.path.exists('drive/MyDrive/models'):
  os.makedirs('drive/MyDrive/models')

In [ ]:
print("torch using "+str(torch.get_num_threads())+" threads")

#SERIAL_EXEC = xmp.MpSerialExecutor()
device="cuda"
data_dir="drive/MyDrive/data/"
batch_size = 16

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased").to(device)


In [ ]:
train_dataset=load_dataset('json', data_files=data_dir+"train.json")["train"]
eval_dataset=load_dataset('json', data_files=data_dir+"test.json")["train"]

In [ ]:
train_loader = DataLoader(
                train_dataset,
                batch_size=batch_size,
                #collate_fn=data_collator,
                num_workers=4,
                drop_last=True
            )

eval_loader = DataLoader(
                eval_dataset,
                batch_size=batch_size,
                #collate_fn=data_collator,
                num_workers=4,
                drop_last=True
            )



In [ ]:
epochs = 3
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
model_dir="drive/MyDrive/models/"
for i in range(epochs):
  # training step
  model.train()
  for idx, tokenized_examples in enumerate(tqdm(train_loader)):
    optimizer.zero_grad()
    input_ids = torch.stack(tokenized_examples['input_ids'])
    attention_mask = torch.stack(tokenized_examples['attention_mask'])

    input_ids = torch.transpose(input_ids, 0, 1).to(device)
    attention_mask = torch.transpose(attention_mask, 0, 1).to(device)

    labels = tokenized_examples['label'].to(device)

    out = model(input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels)
    
    curr_loss = out["loss"]

    curr_loss.backward()
    optimizer.step()


    if idx %5000==0:
      print("epoch ", i, "batch ", idx, ", training loss: ", curr_loss.detach().to("cpu").numpy())
  

  #evaluation
  model.eval()
  total = 0
  matched = 0
  for idx, tokenized_examples in enumerate(tqdm(eval_loader)):
    input_ids = torch.stack(tokenized_examples['input_ids'])
    attention_mask = torch.stack(tokenized_examples['attention_mask'])

    input_ids = torch.transpose(input_ids, 0, 1).to(device)
    attention_mask = torch.transpose(attention_mask, 0, 1).to(device)
    labels = tokenized_examples['label'].to(device)

    pred = torch.argmax(model(input_ids=input_ids,
                attention_mask=attention_mask)["logits"], dim=1)

    total += len(input_ids)
    matched += (pred == labels).int().sum().cpu().numpy()

  
  print("epoch ", i, "test acc: ", matched/total)
  torch.save(model.state_dict(), model_dir+"classifier")
      
